In [6]:
import os
import pickle
import pandas as pd

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import numpy as np


In [49]:
with open("/neurospin/dico/data/deep_folding/current/datasets/schiz/new_stratification/other_start.pkl", 'rb') as file:
    dico = pickle.load(file)

for subset in ['train', 'validation', 'test_intra', 'test']:
    df = dico[subset].participant_id
    df.to_csv(f"/neurospin/dico/data/deep_folding/current/datasets/schiz/new_stratification/{subset}_subjects.csv",
              index=False, header=False)

In [48]:
dico['train']

,participant_id,session,study
0,ESOC10104,1,PRAGUE
1,NM2020,1,SCHIZCONNECT-VIP
2,or130001,1,SCHIZCONNECT-VIP
3,ESOC10112,1,PRAGUE
4,CC6287,1,SCHIZCONNECT-VIP
...,...,...,...
913,HC027,1,CANDI
914,HC010,1,CANDI
915,SS086,1,CANDI
916,HC023,1,CANDI


In [16]:
# Studies
studies = ["schizconnect-vip", "bsnip", "cnp", "candi"]
# Data folders
root = os.path.join("/", "neurospin/psy_sbox/analyses/202205_predict_neurodev/data/root")
_root = os.path.join(root, "morphologist")
# Metadata csv
pd_files = "%s_t1mri_skeleton_participants.csv"
metadata_df = pd.concat([pd.read_csv(os.path.join(_root, pd_files % db)) 
                for db in studies], ignore_index=True, sort=False)
# Little hack
metadata_df.loc[metadata_df['session'].isna(), 'session'] = 1
metadata_df.loc[metadata_df['session'].isin(['v1', 'V1']), 'session'] = 1
metadata_df["session"] = metadata_df["session"].astype(int)
# Add set
dico_scheme = pickle.load(open(os.path.join(root, f"train_val_test_test-intra_scz_stratified.pkl"), "rb"))
unique_keys = ["participant_id", "session", "study"]
metadata_df["set"] = None
for split, scheme in dico_scheme.items():
    scheme['session'] = scheme['session'].astype(int)
    source_keys = metadata_df[unique_keys].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
    assert len(set(source_keys)) == len(source_keys), f"Multiple identique identifiers found"
    target_keys = scheme[unique_keys].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
    mask = source_keys.isin(target_keys).values.astype(bool)
    
    metadata_df.loc[mask, "set"] = split

metadata_df

schiz = metadata_df[metadata_df.diagnosis.isin(['schizophrenia', 'control'])]
schiz = schiz.reset_index(drop=True)
schiz

,participant_id,ni_path,session,run,sex,age,diagnosis,study,site,medication,...,stopsignal,T1w,taskswitch,ScannerSerialNumber,ghost_NoGhost,Subject,Hand,YOB,MR Sessions,set
0,NM3356,/neurospin/psy_sbox/analyses/202205_predict_ne...,1,1,0.0,24.0,schizophrenia,SCHIZCONNECT-VIP,WUSTL,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,validation
1,CH8095a,/neurospin/psy_sbox/analyses/202205_predict_ne...,1,1,1.0,30.0,control,SCHIZCONNECT-VIP,NU,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train
2,CH7131b,/neurospin/psy_sbox/analyses/202205_predict_ne...,1,1,1.0,40.0,control,SCHIZCONNECT-VIP,NU,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,test_intra
3,NM1023,/neurospin/psy_sbox/analyses/202205_predict_ne...,1,1,0.0,23.0,schizophrenia,SCHIZCONNECT-VIP,WUSTL,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train
4,CC8726,/neurospin/psy_sbox/analyses/202205_predict_ne...,1,1,1.0,22.0,control,SCHIZCONNECT-VIP,WUSTL,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1279,SS099,/neurospin/psy_sbox/analyses/202205_predict_ne...,1,1,0.0,0.0,schizophrenia,CANDI,CANDI,NaN,...,NaN,NaN,NaN,NaN,NaN,SS_099,R,NaN,1.0,train
1280,HC012,/neurospin/psy_sbox/analyses/202205_predict_ne...,1,1,1.0,0.0,control,CANDI,CANDI,NaN,...,NaN,NaN,NaN,NaN,NaN,HC_012,R,NaN,1.0,validation
1281,HC010,/neurospin/psy_sbox/analyses/202205_predict_ne...,1,1,0.0,0.0,control,CANDI,CANDI,NaN,...,NaN,NaN,NaN,NaN,NaN,HC_010,R,NaN,1.0,validation
1282,HC001,/neurospin/psy_sbox/analyses/202205_predict_ne...,1,1,1.0,0.0,control,CANDI,CANDI,NaN,...,NaN,NaN,NaN,NaN,NaN,HC_001,R,NaN,1.0,train
